## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-02-15-49-47 +0000,nyt,Search in Montana Continues for Suspect in Fat...,https://www.nytimes.com/2025/08/02/us/montana-...
1,2025-08-02-15-47-00 +0000,wsj,Berkshire’s Quarterly Earnings Drop on Insuran...,https://www.wsj.com/business/earnings/berkshir...
2,2025-08-02-15-42-27 +0000,nyt,A Fish Falls From the Sky and Sparks a Brush F...,https://www.nytimes.com/2025/08/02/world/canad...
3,2025-08-02-15-31-46 +0000,nypost,Trump demurs on pardoning disgraced former Rep...,https://nypost.com/2025/08/02/us-news/presiden...
4,2025-08-02-15-26-07 +0000,nyt,Protesters in Thailand Denounce Prime Minister...,https://www.nytimes.com/2025/08/02/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,59
394,new,17
104,tariffs,12
59,crash,12
365,california,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
154,2025-08-01-22-33-00 +0000,wsj,President Trump fired the top Bureau of Labor ...,https://www.wsj.com/politics/policy/trump-orde...,120
158,2025-08-01-22-27-28 +0000,nypost,Trump admin fires back at claims Clinton plan ...,https://nypost.com/2025/08/01/us-news/trump-ad...,111
162,2025-08-01-22-08-04 +0000,nypost,Trump fires Biden-appointed Bureau of Labor St...,https://nypost.com/2025/08/01/us-news/trump-fi...,109
179,2025-08-01-21-26-41 +0000,latimes,"A raging Trump digs in on his trade war, fires...",https://www.latimes.com/world-nation/story/202...,108
166,2025-08-01-21-57-00 +0000,wsj,The Trump administration is blocking funding f...,https://www.wsj.com/politics/policy/trump-cdc-...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
154,120,2025-08-01-22-33-00 +0000,wsj,President Trump fired the top Bureau of Labor ...,https://www.wsj.com/politics/policy/trump-orde...
191,53,2025-08-01-20-59-00 +0000,wsj,"Ghislaine Maxwell, the former associate of con...",https://www.wsj.com/politics/policy/epstein-as...
158,52,2025-08-01-22-27-28 +0000,nypost,Trump admin fires back at claims Clinton plan ...,https://nypost.com/2025/08/01/us-news/trump-ad...
166,45,2025-08-01-21-57-00 +0000,wsj,The Trump administration is blocking funding f...,https://www.wsj.com/politics/policy/trump-cdc-...
21,42,2025-08-02-13-58-07 +0000,nypost,Illegal migrant — not deported despite multipl...,https://nypost.com/2025/08/02/us-news/allegedl...
266,41,2025-08-01-15-57-21 +0000,cbc,Viable trade deal wasn't on the table ahead of...,https://www.cbc.ca/news/politics/reaction-trum...
5,41,2025-08-02-15-17-07 +0000,bbc,Police find getaway car of army veteran suspec...,https://www.bbc.com/news/articles/c4gz9ep0wnno...
118,41,2025-08-02-01-28-00 +0000,wsj,"After months of mixed signals, official data s...",https://www.wsj.com/economy/the-wild-week-in-t...
244,40,2025-08-01-17-58-15 +0000,nypost,"Anti-Israel NYC Councilwoman Tiffany Caban, do...",https://nypost.com/2025/08/01/us-news/proteste...
254,40,2025-08-01-16-49-02 +0000,nypost,"Trump reveals ‘almost 20,000 Russian soldiers ...",https://nypost.com/2025/08/01/us-news/trump-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
